In [ ]:
source(paste0(dirname(getwd()),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))

### 0 - Read and pathways annotation file

In [ ]:
annotation <- fread(paste0(dirname(getwd()), "/util/annotation/pathways.csv"))
perplexity <- fread(paste0(dirname(getwd()), "/util/annotation/pathways_perplexity.csv"))

In [ ]:
mapping <- list(
  "angiogenesis" = "ANGIOGENESIS",
  "apoptosis" = "APOPTOSIS",
  "cell cycle" = "CELL_CYCLE",
  "chromatin remodeling/epigenetic regulation" = "CHROMATIN",
  "dna damage response" = "DDR",
  "metabolism" = "METABOLISM",
  "hedgehog" = "HEDGEHOG",
  "hippo" = "HIPPO",
  "pd-1/pd-l1/ctla4" = "IMMUNE_EVASION",
  "jak/stat" = "JAK",
  "mapk/erk" = "MAPK",
  "myc" = "MYC",
  "nf-kb" = "NFKB",
  "notch" = "NOTCH",
  "nrf2" = "NRF2",
  "unknown" = "OTHER",
  "pi3k" = "PI3K",
  "dna damage repair" = "DDR",
  "tgfb" = "TGFB",
  "tp53" = "TP53",
  "wnt" = "WNT", 
  "rtk/ras" = "RTK_RAS", 
  "hormone receptor" = "HR", 
  "emt/metastasis" = "EMT_MET",
  "ubiquitination/proteostasis" = "UBQ_PROTEASIS",
  "spliceosome/splicing" = "SPLICING",
  "autophagy" = "AUTOPHAGY",
  "nf-κb" = "NFKB",
  'antigen presentation' = "ANTIGEN_PRESENTATION", 
  'tcr signaling pathway' = "TCR_SIGNALLING", 
  'interferon signaling' = "INTERFERON"    
)

In [ ]:
annotation_perplexity = 
perplexity %>% 
 rw() %>% 
 mu( perplexity_pathway = mapping[[main_pathway]]) %>% 
 ug() %>% 
 se(gene, perplexity_pathway, immune_evasion)

### 1 - Drivers Prep

- Read the dirvers from PURPLE

In [ ]:
drivers <- 
fread( paste0(TMP_DIR, "drivers.csv")) %>% 
  lj(annotation, by = "gene") %>% 
  lj(annotation_perplexity, by = "gene") %>% 
  mu(likelihood = ifelse(category == "TSG" & likelihoodMethod == "DISRUPTION", 1, driverLikelihood)) %>% 
  fi(likelihood > .8)  

- Configure Drivers for biomarker

In [ ]:
drivers_ready <- 
drivers %>% 
  transmute(sampleId, gene = paste0("driver_", gene)) %>% 
  unique() %>% 
  mutate(driver = 1) %>% 
  spread(gene, driver)

drivers_ready[is.na(drivers_ready)] <- 0

- Prepare Pathways

In [ ]:
pathways_ready <- 
drivers %>% 
 gb(sampleId, pathway) %>% 
 su(tot = n()) %>% 
 ug() %>% 
 tm(sampleId, pathway = paste0("drivers_pathway_", pathway), tot) %>% 
 unique() %>% 
 spread(pathway, tot)

pathways_ready[is.na(pathways_ready)] <- 0

- Perplexity Pathways

In [ ]:
perplexity_pathways_ready <- 
drivers %>% 
 gb(sampleId, perplexity_pathway) %>% 
 su(tot = n()) %>% 
 ug() %>% 
 tm(sampleId, pathway = paste0("drivers_pathway_perplexity_", perplexity_pathway), tot) %>% 
 unique() %>% 
 spread(pathway, tot)

perplexity_pathways_ready[is.na(perplexity_pathways_ready)] <- 0

- Totals

In [ ]:
total_drivers <- drivers %>% gb(sampleId) %>% su(drivers_total = n()) 
total_pathways <- drivers %>% gb(sampleId) %>% su(drivers_pathway_total = n_distinct(pathway))
total_pathways_perpexlity <- drivers %>% gb(sampleId) %>% su(drivers_pathway_perplexity_total = n_distinct(perplexity_pathway))
total_immune_evasion <- drivers %>% gb(sampleId) %>% su(drivers_immune_evasion = sum(immune_evasion == "Yes"))

- Juntos

In [ ]:
together <- 
drivers_ready %>% 
 ij(pathways_ready, by = "sampleId") %>% 
 ij(total_drivers, by = "sampleId") %>% 
 ij(total_pathways, by = "sampleId") %>% 
 ij(total_pathways_perpexlity, by = "sampleId") %>% 
 ij(total_immune_evasion, by = "sampleId")

### 2 - Send it!

In [ ]:
fwrite(together, paste0(READY_DIR, "drivers_ready.csv"))